In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import getpass
import pdvega
import plotly.graph_objs as go

from plotly.offline import iplot, init_notebook_mode
import plotly.io as pio
from plotly.graph_objs import *

# for configuring connection 
from configobj import ConfigObj
import os

%matplotlib inline


import os


from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split

#configure the notebook for use in offline mode
init_notebook_mode(connected=True)

In [2]:
df2= pd.read_csv("analysis.csv")

In [3]:
df2.head()

,Unnamed: 0,hospitalid,sodium,electivesurgery,vent,dialysis,gcs,urine,wbc,temperature,...,m11_True,m12_True,m13_True,m14_True,m15_True,m16_True,m17_True,m18_True,m19_True,m20_True
0,0,59.0,139.0,-1.0,0.0,0.0,15.0,-1.0,14.7,36.1,...,1,0,0,1,1,0,0,0,1,0
1,1,73.0,134.0,-1.0,0.0,0.0,13.0,-1.0,14.1,39.3,...,1,0,0,1,1,0,0,0,1,0
2,2,73.0,-1.0,1.0,1.0,0.0,15.0,-1.0,8.0,34.8,...,0,0,1,0,0,1,0,1,0,0
3,3,63.0,137.0,-1.0,0.0,0.0,15.0,-1.0,10.9,36.6,...,1,0,1,1,1,0,0,1,1,0
4,4,63.0,135.0,-1.0,0.0,0.0,15.0,-1.0,5.9,35.0,...,0,0,1,0,0,0,0,1,0,0


In [4]:
del df2['hospitalid']

df2 = df2.drop(df2.columns[[0]], axis=1)

In [5]:
df2.shape

(95148, 83)

**We moved all the pre-processing including splitting>imputation>Standardization to the CV iterations**

In [6]:
cols_to_norm=['gcs', 'urine', 'wbc', 'sodium',
       'temperature', 'respiratoryrate', 'heartrate', 'meanbp', 'creatinine',
       'ph', 'hematocrit', 'albumin', 'pao2', 'pco2', 'bun', 'glucose',
       'bilirubin', 'fio2', 'age', 'offset']

X=df2.drop('destcopy', 1)
y=df2['destcopy']
df_cols = list(X)     #fancy impute removes column names.

In [8]:
# Load in our libraries
import pandas as pd
import numpy as np
import re
import sklearn
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

import warnings
warnings.filterwarnings('ignore')

# Going to use these 5 base models for the stacking
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression

In [9]:
from sklearn.model_selection import StratifiedKFold

In [10]:

classes=['Death','Home','Nursing Home','Rehabilitation']

kf_m = StratifiedKFold(n_splits=10)



# Class to extend the Sklearn classifier
class SklearnHelper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)

    def fit(self,x,y):
        return self.clf.fit(x,y)

    def feature_importances(self,x,y):
        return(self.clf.fit(x,y).feature_importances_)
    



#-------------------------------------------------------------




#------------------------------------------

rf_params = {
    'n_jobs': -1,
    'n_estimators': 400,
     'warm_start': True, 
     #'max_features': 0.2,
    'max_depth': 30,
    'min_samples_leaf': 2,
    'max_features' : 0.8,
    'verbose': 0,
     'criterion':'gini'
}


# Extra Trees Parameters
et_params = {
    'n_jobs': -1,
    'n_estimators':500,
    #'max_features': 0.5,
    'max_depth': 8,
    'min_samples_leaf': 2,
    'verbose': 0
}

# AdaBoost parameters
ada_params = {
    'n_estimators': 500,
    'learning_rate' : 0.75
}

# Gradient Boosting parameters
gb_params = {
    'n_estimators': 500,
     #'max_features': 0.2,
    'max_depth': 5,
    'min_samples_leaf': 2,
    'verbose': 0
}



# Support Vector Classifier parameters 
lr_params = {
    'penalty' : 'l1',
    'tol' : 6.75e-05,
    'C' : 2.5,
    'max_iter': 66
    }



**Random Forest**

In [11]:
from collections import Counter

In [12]:
from sklearn.model_selection import KFold
from sklearn import preprocessing
from imblearn.over_sampling import SMOTENC
from sklearn.metrics import f1_score
from yellowbrick.classifier import ROCAUC
from sklearn.linear_model import LogisticRegression
from numpy import loadtxt
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold
import io 



for fold, (train_index, test_index) in enumerate(kf_m.split(X,y), 1):
    X_train = X.iloc[train_index]
    y_train = y.iloc[train_index]  # Based on your code, you might need a ravel call here, but I would look into how you're generating your y
    X_test = X.iloc[test_index]
    y_test = y.iloc[test_index]  # See comment on ravel and  y_train
    
    
#------------------------------Standardize Testing Set------------------------------------
    
    std_scale = preprocessing.StandardScaler().fit(X_train[cols_to_norm])
    X_train[cols_to_norm] = std_scale.transform(X_train[cols_to_norm])
    X_test[cols_to_norm] = std_scale.transform(X_test[cols_to_norm])
#------------------------------------------------------------------------------------------

 # Hyperparameters are optimized using hyperopt



# Class to extend XGboost classifer
    sm = SMOTENC(random_state=50, categorical_features=[1,2,3,22,23,24,25,26,27,28,29,30,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61, 62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81])
    X_train_oversampled, y_train_oversampled = sm.fit_sample(X_train, y_train)
    print(sorted(Counter(y_train_oversampled).items()))
    
# --------------- Let's Start the fun ------------------------

    # Some useful parameters which will come in handy later on
    ntrain = X_train_oversampled.shape[0]
    print(ntrain)
    ntest = X_test.shape[0]
    SEED = 0 # for reproducibility
    # set folds for out-of-fold prediction
    #kf = KFold(ntrain, n_split=5, random_state=SEED)
    
    def get_oof(clf, x_train, y_train, x_test):
        oof_train = np.zeros((ntrain,))
        oof_test = np.zeros((ntest,))
        oof_test_skf = np.empty((10, ntest))


        for i, (train_index, test_index) in enumerate(kf_m.split(x_train, y_train)):
            x_tr = x_train[train_index]
            y_tr = y_train[train_index]
            x_te = x_train[test_index]

            clf.train(x_tr, y_tr)
            
            oof_train[test_index] = clf.predict(x_te)
            oof_test_skf[i, :] = clf.predict(x_test)

        oof_test[:] = oof_test_skf.mean(axis=0)
        return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)
    
        # Create 5 objects that represent our 4 models
    #rf = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
    et = SklearnHelper(clf=ExtraTreesClassifier, seed=SEED, params=et_params)
    #ada = SklearnHelper(clf=AdaBoostClassifier, seed=SEED, params=ada_params)
    gb = SklearnHelper(clf=GradientBoostingClassifier, seed=SEED, params=gb_params)
    #lr = SklearnHelper(clf=LogisticRegression, seed=SEED, params=lr_params)

    #------------------------------------------
    # Create our OOF train and test predictions. These base results will be used as new features
    et_oof_train, et_oof_test = get_oof(et, X_train_oversampled, y_train_oversampled, X_test) # Extra Trees
    #rf_oof_train, rf_oof_test = get_oof(rf,X_train_oversampled, y_train_oversampled, X_test) # Random Forest
    #ada_oof_train, ada_oof_test = get_oof(ada, X_train_oversampled, y_train_oversampled, X_test) # AdaBoost 
    gb_oof_train, gb_oof_test = get_oof(gb,X_train_oversampled, y_train_oversampled, X_test) # Gradient Boost
    #lr_oof_train, lr_oof_test = get_oof(lr,X_train_oversampled, y_train_oversampled, X_test) # Support Vector Classifier

    print("Training is complete")



    #rf_features = rf.feature_importances(X_train_oversampled,y_train_oversampled).tolist()
    et_features = et.feature_importances(X_train_oversampled, y_train_oversampled).tolist()
    #ada_features = ada.feature_importances(X_train_oversampled, y_train_oversampled).tolist()
    gb_features = gb.feature_importances(X_train_oversampled, y_train_oversampled).tolist()
    #lr_features=(map(abs,lr_features)) / (abs(lr_fit.coef_).max())



    cols = df2.drop('destcopy', 1).columns.values
        # Create a dataframe with features
    feature_dataframe = pd.DataFrame( {'features': cols,
         
         'Extra Trees  feature importances': et_features,
         
         'Gradient Boost feature importances': gb_features,
         #'LR feature importances': lr_features
        })




    # Create a dataframe with features




    # Scatter plot 
    trace = go.Scatter(
        y = feature_dataframe['Extra Trees  feature importances'].values,
        x = feature_dataframe['features'].values,
        mode='markers',
        marker=dict(
            sizemode = 'diameter',
            sizeref = 1,
            size = 25,
    #       size= feature_dataframe['AdaBoost feature importances'].values,
            #color = np.random.randn(500), #set color equal to a variable
            color = feature_dataframe['Extra Trees  feature importances'].values,
            colorscale='Portland',
            showscale=True
        ),
        text = feature_dataframe['features'].values
    )
    data = [trace]

    layout= go.Layout(
        autosize= True,
        title= 'Extra Trees Feature Importance',
        hovermode= 'closest',
    #     xaxis= dict(
    #         title= 'Pop',
    #         ticklen= 5,
    #         zeroline= False,
    #         gridwidth= 2,
    #     ),
        yaxis=dict(
            title= 'Feature Importance',
            ticklen= 5,
            gridwidth= 2
        ),
        showlegend= False
    )
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig,filename='scatter2010')



    # Scatter plot 
    trace = go.Scatter(
        y = feature_dataframe['Gradient Boost feature importances'].values,
        x = feature_dataframe['features'].values,
        mode='markers',
        marker=dict(
            sizemode = 'diameter',
            sizeref = 1,
            size = 25,
    #       size= feature_dataframe['AdaBoost feature importances'].values,
            #color = np.random.randn(500), #set color equal to a variable
            color = feature_dataframe['Gradient Boost feature importances'].values,
            colorscale='Portland',
            showscale=True
        ),
        text = feature_dataframe['features'].values
    )
    data = [trace]

    layout= go.Layout(
        autosize= True,
        title= 'Gradient Boosting Feature Importance',
        hovermode= 'closest',
    #     xaxis= dict(
    #         title= 'Pop',
    #         ticklen= 5,
    #         zeroline= False,
    #         gridwidth= 2,
    #     ),
        yaxis=dict(
            title= 'Feature Importance',
            ticklen= 5,
            gridwidth= 2
        ),
        showlegend= False
    )
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig,filename='scatter2010')

    feature_dataframe['mean'] = feature_dataframe.mean(axis= 1) # axis = 1 computes the mean row-wise
    feature_dataframe.head(3)

    yv = feature_dataframe['mean'].values
    x = feature_dataframe['features'].values
    data = [go.Bar(
                x= x,
                y= yv,
                width = 0.5,
                marker=dict(
                   color = feature_dataframe['mean'].values,
                colorscale='Portland',
                showscale=True,
                reversescale = False
                ),
                opacity=0.6
            )]

    layout= go.Layout(
        autosize= True,
        title= 'Barplots of Mean Feature Importance',
        hovermode= 'closest',
    #     xaxis= dict(
    #         title= 'Pop',
    #         ticklen= 5,
    #         zeroline= False,
    #         gridwidth= 2,
    #     ),
        yaxis=dict(
            title= 'Feature Importance',
            ticklen= 5,
            gridwidth= 2
        ),
        showlegend= False
    )
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig, filename='bar-direct-labels')




    base_predictions_train = pd.DataFrame( {
     'ExtraTrees': et_oof_train.ravel(),
     
     'GradientBoost': gb_oof_train.ravel(),
     #'LR': lr_oof_train.ravel()
    })
    base_predictions_train.head()

    data = [
    go.Heatmap(
        z= base_predictions_train.astype(float).corr().values ,
        x=base_predictions_train.columns.values,
        y= base_predictions_train.columns.values,
          colorscale='Viridis',
            showscale=True,
            reversescale = True
        )
    ]
    py.iplot(data, filename='labelled-heatmap')

    x_train = np.concatenate(( et_oof_train,gb_oof_train), axis=1)
    x_test = np.concatenate(( et_oof_test, gb_oof_test), axis=1)
    
    gbm = RandomForestClassifier().fit(x_train,y_train_oversampled)
    y_pred = gbm.predict(x_test)
    visualizer = ROCAUC(gbm, classes=classes)
    visualizer.fit(x_train, y_train_oversampled)  # Fit the training data to the visualizer
    visualizer.score(x_test, y_test)  # Evaluate the model on the test data
    visualizer.poof("Ensembel_{}.pdf".format(fold), clear_figure=True) 
    print(f'For fold {fold}:')
    print(f'Accuracy: {gbm.score(x_test, y_test)}')
    f1=f1_score(y_test, y_pred, average='micro')
    print(f'f-score: {f1}')
    print(classification_report_imbalanced(y_test, y_pred))
    K= classification_report_imbalanced(y_test, y_pred)
    df = pd.read_fwf(io.StringIO(K))
    df.loc["1":"1","pre":"sup"].to_csv("RF-Ensemble-D.csv" , sep=',', encoding='utf-8', doublequote=False, index=False, mode="a", header=False)
    df.loc["2":"2","pre":"sup"].to_csv("RF-Ensemble-H.csv" , sep=',', encoding='utf-8', doublequote=False, index=False, mode="a", header=False)
    df.loc["3":"3","pre":"sup"].to_csv("RF-Ensemble-N.csv" , sep=',', encoding='utf-8', doublequote=False, index=False, mode="a", header=False)
    df.loc["4":"4","pre":"sup"].to_csv("RF-Ensemble-R.csv" , sep=',', encoding='utf-8', doublequote=False, index=False, mode="a", header=False)
    df.iloc[6:7,:].to_csv("RF-Ensemble-avg.csv" , sep=',', encoding='utf-8', doublequote=False, index=False, mode="a", header=False)
    
    

[(1, 59596), (2, 59596), (3, 59596), (4, 59596)]
238384
Training is complete


For fold 1:
Accuracy: 0.7351828499369483
f-score: 0.7351828499369484


NameError: name 'classification_report_imbalanced' is not defined

<Figure size 576x396 with 0 Axes>

In [ ]:
len(y_test)

In [ ]:
list(map(abs,lr_features))

In [ ]:
visualizer


In [ ]:
fig.write_image("images/fig1.png")

In [ ]:
#lr_fit= lr.fit(X_train_oversampled, y_train_oversampled).tolist()
lr_features = lr_fit.coef_
len(list(lr_features.flat))

In [ ]:
  
    
    model = AdaBoostClassifier() 
    model.fit(X_train_oversampled, y_train_oversampled)  
    y_pred = model.predict(X_test.values)
    visualizer = ROCAUC(model, classes=classes)
    visualizer.fit(X_train_oversampled, y_train_oversampled)  # Fit the training data to the visualizer
    visualizer.score(X_test.values, y_test)  # Evaluate the model on the test data
    visualizer.poof("Ada_Indicator_Replace_{}.pdf".format(fold), clear_figure=True) 
    print(f'For fold {fold}:')
    print(f'Accuracy: {model.score(X_test.values, y_test)}')
    f1=f1_score(y_test, y_pred, average='micro')
    print(f'f-score: {f1}')
    print(classification_report_imbalanced(y_test, y_pred))
    
    #

    

In [ ]:
 feature_dataframe['mean'] = feature_dataframe.mean(axis= 1) # axis = 1 computes the mean row-wise
    feature_dataframe.head(3)
    
    y = feature_dataframe['mean'].values
    x = feature_dataframe['features'].values
    data = [go.Bar(
                x= x,
                 y= y,
                width = 0.5,
                marker=dict(
                   color = feature_dataframe['mean'].values,
                colorscale='Portland',
                showscale=True,
                reversescale = False
                ),
                opacity=0.6
            )]

    layout= go.Layout(
        autosize= True,
        title= 'Barplots of Mean Feature Importance',
        hovermode= 'closest',
    #     xaxis= dict(
    #         title= 'Pop',
    #         ticklen= 5,
    #         zeroline= False,
    #         gridwidth= 2,
    #     ),
        yaxis=dict(
            title= 'Feature Importance',
            ticklen= 5,
            gridwidth= 2
        ),
        showlegend= False
    )
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig, filename='bar-direct-labels')
    
    base_predictions_train = pd.DataFrame( {
     'ExtraTrees': et_oof_train.ravel(),
     'GradientBoost': gb_oof_train.ravel()
    })
    base_predictions_train.head()
    
    data = [
    go.Heatmap(
        z= base_predictions_train.astype(float).corr().values ,
        x=base_predictions_train.columns.values,
        y= base_predictions_train.columns.values,
          colorscale='Viridis',
            showscale=True,
            reversescale = True
    )
    ]
    py.iplot(data, filename='labelled-heatmap')
    
    #-------------------------------------------------------------------------------------
    x_train = np.concatenate(( et_oof_train, rf_oof_train, ada_oof_train, gb_oof_train, lr_oof_train), axis=1)
    x_test = np.concatenate(( et_oof_test, rf_oof_test, ada_oof_test, gb_oof_test, lr_oof_test), axis=1)
    
    gbm = xgb.XGBClassifier(
    #learning_rate = 0.02,
     n_estimators= 2000,
     max_depth= 4,
     min_child_weight= 2,
     #gamma=1,
     gamma=0.9,                        
     subsample=0.8,
     colsample_bytree=0.8,
     objective= 'binary:logistic',
     nthread= -1,
     scale_pos_weight=1).fit(x_train, y_train_oversampled)
    predictions = gbm.predict(x_test)
    

In [ ]:
len(lr_features)

In [ ]:
cols = df2.drop('destcopy', 1).columns.values

In [ ]:
import plotly.graph_objects as go

In [ ]:
fig.show()

In [ ]:
import numpy as np    
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import pandas as pd
import matplotlib.pyplot as plt

x1 = np.random.randn(100)
x2 = np.random.randn(100)
x3 = np.random.randn(100)

#Make difference in feature dependance
y = (3 + x1 + 2*x2 + 5*x3 + 0.2*np.random.randn()) > 0

X = pd.DataFrame({'x1':x1,'x2':x2,'x3':x3})

#Scale your data
scaler = StandardScaler()
scaler.fit(X) 
X_scaled = pd.DataFrame(scaler.transform(X),columns = X.columns)

clf = LogisticRegression(random_state = 0)
clf.fit(X_scaled, y)

feature_importance = abs(clf.coef_[0])
feature_importance = 100.0 * (feature_importance / feature_importance.max())
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + .5

featfig = plt.figure()
featax = featfig.add_subplot(1, 1, 1)
featax.barh(pos, feature_importance[sorted_idx], align='center')
featax.set_yticks(pos)
featax.set_yticklabels(np.array(X.columns)[sorted_idx], fontsize=8)
featax.set_xlabel('Relative Feature Importance')

plt.tight_layout()   
plt.show()

In [ ]:
feature_importance